In this notebook we will simulate First Passage Percolation on the Two-Dimensional Integer lattice, and illustrate the convergence in measure of the rescaled ball of radius n 1/n*B_n to a fixed shape. 

(TODO: double check) While this convergence was proved in 1990 by Boivin, to the author's best knowledge, the shape that the balls converge _to_ is yet undetermined.

In [11]:
import numpy as np
import random

In [58]:
N = 1

In [118]:
def add_tuple(t1, t2): #python dicts can't have arrays as keys, so we'll use tuples. We'll need to add tuples later.
    return(tuple(map(sum, zip(t1, t2))))

def get_l1_norm(tuple):
    return(np.linalg.norm(np.array(tuple), ord=1))

def get_random_weight():
    return(random.choice([1,2]))

def initialize_points(N):
    '''Use L-infinity norm to initialize relevant integer points'''
    pts = {(x,y) for x in range(-N, N+1) for y in range(-N,N+1)}
    return(pts)

def initialize_edges_and_weights(N):
    pts = initialize_points(N)
    edges = {}
    for pt in pts:
        for unit_vector in [(1,0), (0,1)]:
            if add_tuple(pt, unit_vector) in pts:
                edges[(pt, add_tuple(pt, unit_vector))] = get_random_weight()
    return(pts, edges)

def get_costs(N):
    #assign a cost to each point by iterating over L1 norm
    pts, edges = initialize_edges_and_weights(N)
    costs = {}
    
    costs[(0,0)] = 0
    for k in range(1,N+1):
        Bk = [pt for pt in pts if get_l1_norm(pt)==k]
        for pt in Bk:
            #find euclidean geodesic extensions to pt by checking if cost of preceding point is calculated
            #find associated costs. take min.
            #this is avoiding hard-coding logic about the quadrant the current point is in
            curr_costs = []
            if get_l1_norm(add_tuple(pt, (1,0))) == k-1:
                incremental_cost = edges[(pt, add_tuple(pt, (1,0)))]
                total_cost = costs[add_tuple(pt, (1,0))] + incremental_cost
                curr_costs.append(total_cost)
            if get_l1_norm(add_tuple(pt, (0,1))) == k-1:
                incremental_cost = edges[(pt, add_tuple(pt, (0,1)))]
                total_cost = costs[add_tuple(pt, (0,1))] + incremental_cost
                curr_costs.append(total_cost)
            if get_l1_norm(add_tuple(pt, (-1,0))) == k-1:
                incremental_cost = edges[(add_tuple(pt, (-1,0)), pt)]
                total_cost = costs[add_tuple(pt, (-1,0))] + incremental_cost
                curr_costs.append(total_cost)
            if get_l1_norm(add_tuple(pt, (0,-1))) == k-1:
                incremental_cost = edges[(add_tuple(pt, (0,-1)), pt)]
                total_cost = costs[add_tuple(pt, (0,-1))] + incremental_cost
                curr_costs.append(total_cost)
            curr_cost = min(curr_costs)
            costs[pt] = curr_cost
    return(edges, costs)

In [119]:
#26 seconds to compute costs for N=100 on one sample. Should be acceptable.
edges, costs = get_costs(N=10)
print(costs)

{(0, 0): 0, (-1, 0): 1, (1, 0): 2, (0, -1): 2, (0, 1): 2, (-1, 1): 3, (-2, 0): 2, (0, -2): 4, (0, 2): 4, (2, 0): 4, (-1, -1): 3, (1, -1): 3, (1, 1): 4, (3, 0): 6, (-2, 1): 3, (2, -1): 4, (0, 3): 5, (0, -3): 5, (2, 1): 5, (-2, -1): 3, (-1, -2): 4, (-1, 2): 4, (1, -2): 4, (-3, 0): 4, (1, 2): 6, (0, -4): 6, (3, 1): 7, (2, -2): 5, (2, 2): 7, (-3, -1): 5, (0, 4): 6, (4, 0): 7, (-4, 0): 5, (3, -1): 6, (-1, 3): 5, (-2, -2): 4, (-3, 1): 5, (-2, 2): 5, (1, 3): 7, (-1, -3): 5, (1, -3): 5, (0, 5): 7, (-4, -1): 6, (0, -5): 7, (2, 3): 8, (-5, 0): 6, (-1, -4): 6, (2, -3): 6, (-1, 4): 7, (1, -4): 6, (-3, -2): 6, (-3, 2): 6, (4, 1): 9, (1, 4): 7, (-4, 1): 6, (3, -2): 6, (3, 2): 8, (-2, 3): 7, (5, 0): 8, (-2, -3): 6, (4, -1): 7, (4, 2): 10, (-4, -2): 7, (-5, 1): 7, (-4, 2): 7, (6, 0): 10, (-2, -4): 7, (-1, 5): 8, (5, -1): 8, (-3, 3): 7, (-1, -5): 7, (-2, 4): 8, (1, 5): 9, (1, -5): 7, (-3, -3): 7, (0, -6): 9, (3, 3): 10, (-5, -1): 7, (2, -4): 8, (3, -3): 8, (5, 1): 9, (2, 4): 8, (-6, 0): 8, (0, 6): 9, (

1

In [75]:
x = (2,2)
np.linalg.norm(np.array(x), ord=1)

4.0

In [66]:
edges = initialize_edges_and_weights(3)
print(len(edges))
print(edges)

84
{((3, -2), (3, -1)): 1, ((3, -1), (3, 0)): 1, ((3, 1), (3, 2)): 1, ((-3, -3), (-2, -3)): 1, ((-3, -3), (-3, -2)): 2, ((-3, 0), (-2, 0)): 2, ((-3, 0), (-3, 1)): 1, ((-3, 3), (-2, 3)): 2, ((0, 2), (1, 2)): 1, ((0, 2), (0, 3)): 2, ((1, -3), (2, -3)): 1, ((1, -3), (1, -2)): 2, ((2, 2), (3, 2)): 1, ((2, 2), (2, 3)): 2, ((1, 0), (2, 0)): 1, ((1, 0), (1, 1)): 1, ((1, 3), (2, 3)): 1, ((-2, -2), (-1, -2)): 2, ((-2, -2), (-2, -1)): 1, ((-2, -1), (-1, -1)): 2, ((-2, -1), (-2, 0)): 2, ((-1, -2), (0, -2)): 1, ((-1, -2), (-1, -1)): 2, ((-1, -1), (0, -1)): 1, ((-1, -1), (-1, 0)): 1, ((-2, 1), (-1, 1)): 2, ((-2, 1), (-2, 2)): 2, ((-1, 1), (0, 1)): 1, ((-1, 1), (-1, 2)): 1, ((3, -3), (3, -2)): 2, ((3, 0), (3, 1)): 1, ((-3, 2), (-2, 2)): 1, ((-3, 2), (-3, 3)): 2, ((0, -2), (1, -2)): 1, ((0, -2), (0, -1)): 2, ((0, -1), (1, -1)): 1, ((0, -1), (0, 0)): 1, ((0, 1), (1, 1)): 1, ((0, 1), (0, 2)): 1, ((2, -2), (3, -2)): 2, ((2, -2), (2, -1)): 1, ((2, -1), (3, -1)): 1, ((2, -1), (2, 0)): 2, ((1, 2), (2, 2)):

In [68]:
edges[((0,0),(0,1))]

1

In [69]:
edges

{((3, -2), (3, -1)): 1,
 ((3, -1), (3, 0)): 1,
 ((3, 1), (3, 2)): 1,
 ((-3, -3), (-2, -3)): 1,
 ((-3, -3), (-3, -2)): 2,
 ((-3, 0), (-2, 0)): 2,
 ((-3, 0), (-3, 1)): 1,
 ((-3, 3), (-2, 3)): 2,
 ((0, 2), (1, 2)): 1,
 ((0, 2), (0, 3)): 2,
 ((1, -3), (2, -3)): 1,
 ((1, -3), (1, -2)): 2,
 ((2, 2), (3, 2)): 1,
 ((2, 2), (2, 3)): 2,
 ((1, 0), (2, 0)): 1,
 ((1, 0), (1, 1)): 1,
 ((1, 3), (2, 3)): 1,
 ((-2, -2), (-1, -2)): 2,
 ((-2, -2), (-2, -1)): 1,
 ((-2, -1), (-1, -1)): 2,
 ((-2, -1), (-2, 0)): 2,
 ((-1, -2), (0, -2)): 1,
 ((-1, -2), (-1, -1)): 2,
 ((-1, -1), (0, -1)): 1,
 ((-1, -1), (-1, 0)): 1,
 ((-2, 1), (-1, 1)): 2,
 ((-2, 1), (-2, 2)): 2,
 ((-1, 1), (0, 1)): 1,
 ((-1, 1), (-1, 2)): 1,
 ((3, -3), (3, -2)): 2,
 ((3, 0), (3, 1)): 1,
 ((-3, 2), (-2, 2)): 1,
 ((-3, 2), (-3, 3)): 2,
 ((0, -2), (1, -2)): 1,
 ((0, -2), (0, -1)): 2,
 ((0, -1), (1, -1)): 1,
 ((0, -1), (0, 0)): 1,
 ((0, 1), (1, 1)): 1,
 ((0, 1), (0, 2)): 1,
 ((2, -2), (3, -2)): 2,
 ((2, -2), (2, -1)): 1,
 ((2, -1), (3, -1)): 1,
 